In [0]:
import glob
import re
import datetime
import databricks.koalas as pd
from pyspark.sql.functions import to_timestamp, year, month, dayofmonth, hour, minute, second, col, substring_index, length, pow, max, lag, unix_timestamp, avg, mean as _mean, stddev as _stddev, max as _max, min as _min, col, abs

from pyspark.sql.window import Window

In [0]:
%fs
ls /mnt/gmps/delta/just_in_case/

path,name,size
dbfs:/mnt/gmps/delta/just_in_case/B:ACMNIG/,B:ACMNIG/,0
dbfs:/mnt/gmps/delta/just_in_case/B:ACMNPG/,B:ACMNPG/,0
dbfs:/mnt/gmps/delta/just_in_case/B:ACMXIG/,B:ACMXIG/,0
dbfs:/mnt/gmps/delta/just_in_case/B:ACMXPG/,B:ACMXPG/,0
dbfs:/mnt/gmps/delta/just_in_case/B:DCIG/,B:DCIG/,0
dbfs:/mnt/gmps/delta/just_in_case/B:DCPG/,B:DCPG/,0
dbfs:/mnt/gmps/delta/just_in_case/B:GMPS1V/,B:GMPS1V/,0
dbfs:/mnt/gmps/delta/just_in_case/B:GMPS2V/,B:GMPS2V/,0
dbfs:/mnt/gmps/delta/just_in_case/B:GMPS3V/,B:GMPS3V/,0
dbfs:/mnt/gmps/delta/just_in_case/B:GMPS4V/,B:GMPS4V/,0


In [0]:
directory = '/dbfs/mnt/delta/'
relevant = glob.glob1(str(directory),"B*")+glob.glob1(str(directory),"I*")+glob.glob1(str(directory),"E*")
print(relevant)

['B:ACMNIG', 'B:ACMNPG', 'B:ACMXIG', 'B:ACMXPG', 'B:DCIG', 'B:DCPG', 'B:GMPS1V', 'B:GMPS2V', 'B:GMPS3V', 'B:GMPS4V', 'B:GMPSBT', 'B:GMPSFF', 'B:IMAXXG', 'B:IMAXXO', 'B:IMINER', 'B:IMINST', 'B:IMINXG', 'B:IMINXO', 'B:IPHSTC', 'B:LINFRQ', 'B:NGMPS', 'B:PS1VGM', 'B:PS1VGP', 'B:PS2VGM', 'B:PS2VGP', 'B:PS3VGM', 'B:PS3VGP', 'B:PS4VGM', 'B:PS4VGP', 'B:VIMAX', 'B:VIMIN', 'B:VINHBT', 'B:VIPHAS', 'B_ACMNIG', 'B_ACMNPG', 'B_ACMXIG', 'B_ACMXPG', 'B_DCIG', 'B_DCPG', 'B_GMPSBT', 'B_GMPSFF', 'B_IMAXXG', 'B_IMAXXO', 'B_IMINST', 'B_IMINXG', 'B_IMINXO', 'B_IPHSTC', 'B_VIMAX', 'B_VIMIN', 'B_VINHBT', 'B_VIPHAS', 'I:IB', 'I:MDAT40', 'I:MXIB', 'Event0c']

In [0]:
# relevant2 = ['B_IMINXO', 'B_IPHSTC', 'B_VIMAX', 'B_VIMIN', 'B_VINHBT', 'B_VIPHAS']
# relevant3 = ['I:IB', 'I:MDAT40', 'I:MXIB', 'Event0c']
# relevant4 = ['B_ACMXPG']

relevant5 = ['B_GMPSFF']

In [0]:
param = 'B|GMPSSC'
device1 = spark.read.format("delta").load("/mnt/gmps/delta/just_in_case/"+str(param))
display(device1)

time,B|GMPSSC
2019-06-02T05:00:00.098+0000,407.0
2019-06-02T05:09:58.462+0000,407.0
2019-06-02T05:19:56.636+0000,407.0
2019-06-02T05:29:54.755+0000,407.0
2019-06-02T05:39:52.855+0000,407.0
2019-06-02T05:49:50.907+0000,407.0
2019-06-02T05:59:49.880+0000,407.0
2019-06-02T06:09:48.005+0000,407.0
2019-06-02T06:19:46.098+0000,407.0
2019-06-02T06:29:44.282+0000,407.0


In [0]:
# out = open("log.txt", "w")
# out.write("param mean std max min\n")
# out.flush()
print("param mean std max min")

for param in ['B|GMPSSC']:
#for param in ['B:ACMNIG', 'I:IB']:
  if param != 'Event0c':
    device1 = spark.read.format("delta").load("/mnt/gmps/delta/just_in_case/"+str(param))
    
    #w3 = Window.orderBy("file")
    w3 = Window.orderBy("time")
    device1_lag = device1.withColumn("prev_value", lag(device1.time,1).over(w3)).withColumn("diffdevice1", 1000*((unix_timestamp('time')
                         + substring_index('time', '.', -1).cast('long')/pow(10,length(substring_index('time', '.', -1))) 
                         - (unix_timestamp('prev_value') 
                            + substring_index('prev_value', '.', -1).cast('long')/pow(10,length(substring_index('prev_value', '.', -1)))))))
    
    #hertz = device1_lag.where(abs(col("diffdevice1")) <= 100000)
    hertz = device1_lag
    
    df_stats = hertz.select(
              _mean(col('diffdevice1')).alias('mean'),
              _stddev(col('diffdevice1')).alias('std'),
              _max(col('diffdevice1')).alias('max'),
              _min(col('diffdevice1')).alias('min')).collect()

    mean = df_stats[0]['mean']
    std = df_stats[0]['std']
    maxi = df_stats[0]['max']
    mini = df_stats[0]['min']
    
#     out.write("param mean std max min\n")
#     out.flush("{} {} {} {} {}\n".format(param, mean, std, maxi, mini))
    print("{} {} {} {} {}".format(param, mean, std, maxi, mini))
    
  else:
    accel_base = spark.read.format("delta").load("/mnt/delta/Event0c")
    timedf2 = accel_base.select(col("file"), to_timestamp(col("time")).alias("goldTS"))
    
    w2 = Window.orderBy("goldTS")
    event0c_lag = timedf2.withColumn("prev_value", lag(timedf2.goldTS,1).over(w2)).withColumn("diffgoldt", 1000*((unix_timestamp('goldTS')
                         + substring_index('goldTS', '.', -1).cast('long')/pow(10,length(substring_index('goldTS', '.', -1))) 
                         - (unix_timestamp('prev_value') 
                            + substring_index('prev_value', '.', -1).cast('long')/pow(10,length(substring_index('prev_value', '.', -1)))))))
    
    hertz = event0c_lag.where(abs(col("diffgoldt")) <= 100000)
    
    df_stats = hertz.select(
              _mean(col('diffgoldt')).alias('mean'),
              _stddev(col('diffgoldt')).alias('std'),
              _max(col('diffgoldt')).alias('max'),
              _min(col('diffgoldt')).alias('min')).collect()

    mean = df_stats[0]['mean']
    std = df_stats[0]['std']
    maxi = df_stats[0]['max']
    mini = df_stats[0]['min']
    
    print("{} {} {} {} {}".format(param, mean, std, maxi, mini))
    
#     out.write("param mean std max min\n")
#     out.flush("{} {} {} {} {}\n".format(param, mean, std, maxi, mini))

# out.close() 

param mean std max min
B|GMPSSC 697167.3092240304 18768555.798906475 3715289745.000124 74993.99900436401

In [0]:
out = open("test.txt", "w")
out.write("param mean std max min\n")
out.flush()

In [0]:
dbutils.fs.put("/FileStore/my-stuff/my-file.txt", "Contents of my file")